In [1]:
import osmnx as ox
import pandas as pd
from routing import astar_route
import folium
import matplotlib.pyplot as plt
import routing_utils as ru


In [2]:
G = ru.load_graph() # Load Graph can take 2-5 minutes, due to large graph size

In [5]:
origin = (34.044801, -118.279684)
dest = (34.1300, -118.274077)
weather = "Clear"

ru.VisualizeMap(G, origin, dest, weather)

[122702237, 122702233, 123746541, 122702264, 123063526, 21748275, 122656759, 122656753, 122656750, 122639915, 122656748, 122656744, 21300247, 21306068, 122656741, 123520103, 122656738, 11386700826, 122656736, 11386700821, 18166944, 122681219, 60946387, 122450842, 122450742, 122681222, 122681225, 122681227, 122681230, 122681233, 122681235, 122681236, 122681239, 20400296, 14940283, 122681247, 17597297, 6787708196, 122681255, 610178522, 561233387, 341336086, 122657673, 341335956, 122814858, 122814860, 122669779, 605262092, 17597292, 341331047, 1614923794, 122814704, 20954378, 122549922, 1849463223, 122676626, 26277276, 122814707, 26277275, 122814712, 1832241689, 88892737, 15855220, 10583638341, 1832131524, 118507464, 17596252, 118507349, 15027590, 27497861, 365467068, 61083244, 365467065, 365467232, 365467195, 6810351565, 6810351572, 365467527, 111907830, 118507301, 111907772, 1791056914, 1791056851, 1791056867, 1832079939, 1791056883, 1832079955, 1832079956, 25464330, 1832047237, 1832047

In [5]:
def get_top_risky_roads(G, top_n=10):
    risky_edges = []

    for u, v, key, data in G.edges(keys=True, data=True):
        name = data.get("name")
        if isinstance(name, list):
            name = name[0]
        if not name:
            name = "Unnamed Road"

        length = data.get("length", 0)
        cost_risk = data.get("cost_risk", 0)
        base_risk = (cost_risk / length - 1) if length > 0 else 0

        risky_edges.append({
            "road_name": name,
            "risk_score": base_risk,
            "length_m": length,
            "u": u,
            "v": v,
            "key": key
        })

    # Sort by risk score descending
    risky_edges = sorted(risky_edges, key=lambda x: x["risk_score"], reverse=True)

    # Convert to DataFrame for easy viewing
    risky_df = pd.DataFrame(risky_edges[:top_n])
    return risky_df


In [6]:
get_top_risky_roads(G=G, top_n=10)

,road_name,risk_score,length_m,u,v,key
0,Santa Monica Freeway,1.0,517.234416,653721,1613525130,0
1,Santa Monica Freeway,1.0,543.333291,653731,277327278,0
2,Santa Monica Freeway,1.0,626.662780,653744,1614922693,0
3,Santa Monica Freeway,1.0,392.187693,15857120,627837690,0
4,Santa Monica Freeway,1.0,413.097138,18166123,61940547,0
5,Santa Monica Freeway,1.0,872.921681,18166124,24960499,0
6,Santa Monica Freeway,1.0,669.682557,21063602,660529420,0
7,Santa Monica Freeway,1.0,451.680160,21063610,304903916,0
8,Santa Monica Freeway,1.0,593.050932,21098478,1613371627,0
9,Santa Monica Freeway,1.0,263.470648,21098478,38560128,0


In [ ]:
##########################################################################################################################################################################################

In [ ]:
def path_finding(G, origin, dest, use_risk):
    G = ru.get_edge_data(G)
    cost_attr = "cost_risk" if use_risk else "cost_distance"

    #origin = (34.06655181863482, -118.18789017013762)
    #dest = (34.0647417476385, -118.25823027030046)

    if use_risk:
        node_path, edge_path, road_names = astar_route(
            G,
            origin_point=origin,
            destination_point=dest,
            weight=cost_attr
        )
        print(node_path)
        print(f"Number of nodes in path: {len(node_path)}")
        print(f"Number of edges in path: {len(edge_path)}")
        print("Roads to follow:", road_names)

    # Compute fastest route (minimize travel time)
    node_path_fast, edge_path_fast, road_names_fast = astar_route(
        G,
        origin_point=origin,
        destination_point=dest,
        weight="cost_length",
    )
    
    print(node_path_fast)
    print(f"Number of nodes in path (fastest route): {len(node_path_fast)}")
    print(f"Number of edges in path (fastest route): {len(node_path_fast)}")
    print("Roads to follow(fastest route): ", road_names_fast)

    return G, edge_path, edge_path_fast

In [ ]:
def VisualizesMapOld(G, origin, dest):
    G, edge_path, edge_path_fast = ru.path_finding(G, origin, dest)

    # Calculate total travel time for each route (in minutes)
    total_time_risk = sum(G.edges[u, v, key]["cost_time"] for u, v, key in edge_path) * 60
    total_time_fast = sum(G.edges[u, v, key]["cost_time"] for u, v, key in edge_path_fast) * 60

    # Create a folium map centered on the origin
    m = folium.Map(location=origin, zoom_start=13)
    counter = -1
    
     ## Risk Route Coloring ###
    edge_path = ru.path_finding(G)[2]
    for u, v, key in edge_path:
        counter+=1
        data = G.edges[u, v, key]
        if 'geometry' in data:
            line = data['geometry']
            coords = [(lat, lon) for lon, lat in line.coords]
            if counter == 0:
                first_node_coords = coords[0]

        else:
            point_u = (G.nodes[u]['y'], G.nodes[u]['x'])
            point_v = (G.nodes[v]['y'], G.nodes[v]['x'])
            #line = LineString([point_u, point_v])
            coords = [point_u, point_v]
        length = data.get("length", 1.0)
        cost_risk = data.get("cost_risk", length)
        base_risk = (cost_risk / length) - 1
        color = ru.risk_to_color(base_risk)
        folium.PolyLine(
            locations=coords,#[(lat, lon) for lon, lat in line.coords],
            color=color,
            weight=5,
            opacity=0.8,
            popup=f"{data.get('name', 'Unnamed Road')}<br>Risk: {base_risk:.2f}"
        ).add_to(m)
        # Add the start coords to first node line and end coords to last node line
    folium.PolyLine(locations=[origin, first_node_coords],
                    color=first_color,
                    weight=5,
                    opacity=0.8,
                    popup=f"{data.get('name', 'Unnamed Road')}<br>Risk: {base_risk:.2f}"
                    ).add_to(m)
    print('edge_path_end:', edge_path[-1])
    last_u, last_v, last_key = edge_path[-1]
    data = G.edges[last_u, last_v, last_key]

    if 'geometry' in data:
        last_coords = (data['geometry'].coords[-1][1], data['geometry'].coords[-1][0])  # (lat, lon)
    else:
        last_coords = (G.nodes[last_v]['y'], G.nodes[last_v]['x'])  # fallback if no geometry


    edge_path_fast = ru.path_finding(G)[2]
    ### Fastest Route Coloring ###
    for u, v, key in edge_path_fast:
        data = G.edges[u, v, key]
        if 'geometry' in data:
            line = data['geometry']
            coords = [(lat, lon) for lon, lat in line.coords]
            
        else:
            point_u = (G.nodes[u]['y'], G.nodes[u]['x'])
            point_v = (G.nodes[v]['y'], G.nodes[v]['x'])
            #line = LineString([point_u, point_v])
            coords = [point_u, point_v]
        folium.PolyLine(
            locations=coords,#[(lat, lon) for lon, lat in line.coords],
            color="blue",
            weight=3,
            opacity=0.7,
            popup=f"Fastest: {data.get('name', 'Unnamed Road')}"
        ).add_to(m)

    

    folium.PolyLine(locations=[last_coords, dest],
                    color=last_color,
                    weight=5,
                    opacity=0.8,
                    popup=f"{data.get('name', 'Unnamed Road')}<br>Risk: {base_risk:.2f}"
                    ).add_to(m)
    # Add start and end markers
    folium.Marker(origin, tooltip="Start", icon=folium.Icon(color="green")).add_to(m)
    folium.Marker(dest, tooltip="Destination", icon=folium.Icon(color="red")).add_to(m)

    # Add info box with travel times
    info_html = f"""
    <div style="position: fixed; 
                top: 10px; left: 60px; width: 250px; height: 100px; 
                background-color: white; z-index:9999; font-size:14px;
                border:2px solid grey; padding: 10px;">
    <b>Route Time Estimates</b><br>
    <span style='color:black;'>Safest route: </span>{total_time_risk:.1f} min<br>
    <span style='color:blue;'>Fastest route: </span>{total_time_fast:.1f} min
    </div>
    """
    m.get_root().html.add_child(folium.Element(info_html))

    # Risk legend
    legend_html = """
    <div style="position: fixed; 
                top: 10px; left: 280px; width: 150px; height: 100px; 
                background-color: white; z-index:9999; font-size:14px;
                border:2px solid grey; padding: 10px;">
    <b>Risk Level</b><br>
    <span style="color:#00ff00;">●</span> Low<br>
    <span style="color:#ffff00;">●</span> Medium<br>
    <span style="color:#ff0000;">●</span> High
    </div>
    """
    m.get_root().html.add_child(folium.Element(legend_html))

    # Map styles
    folium.TileLayer(titles = "Stamen Terrain").add_to(m)
    folium.TileLayer("CartoDB Positron").add_to(m)
    folium.TileLayer("openstreetmap").add_to(m)
    folium.LayerControl().add_to(m)

    # Save to HTML
    m.save("risky_path.html")
    print("Map saved to risky_path.html")


In [ ]:
origin = (33.974801, -118.279684)
dest = (34.010984, -117.276077)

VisualizesMapOld(G, origin, dest, use_risk=True)

TypeError: VisualizesMap() got an unexpected keyword argument 'use_risk'